In [57]:
import pandas as pd
import numpy as np

In [58]:
city_pop = pd.read_csv("sub-est2021_all.csv",encoding='latin-1')
state_codes = pd.read_csv("state_codes.csv")

city_pop = city_pop[["PLACE","NAME", "STNAME","POPESTIMATE2021","POPESTIMATE2020"]]
city_only = [i.lower().__contains__("city") for i in city_pop["NAME"]] 
city_pop = city_pop[city_only]
city_pop = city_pop[city_pop["POPESTIMATE2021"]>=50000]
city_pop = city_pop.set_axis(['place_code','city', 'state', 'pop_estimate_2021','pop_estimate_2020'], axis='columns', copy = False)
city_pop["city"] = [i.replace(" city","") for i in city_pop["city"]]

state_codes = state_codes[["state","code"]].set_axis(['state', 'state_code'], axis='columns', copy = False)
city_pop = pd.merge(city_pop,state_codes, on = "state", how = "left")
city_pop["city_state"] = city_pop["city"] + ", " + city_pop["state"]
city_pop["city_state_code"] = city_pop["city"] + ", " + city_pop["state_code"]
city_pop["place_state_code"] = [str(i)+j for i,j in zip(city_pop["place_code"],city_pop["state_code"])]
city_pop = city_pop.drop_duplicates(subset = "place_state_code")
city_pop = city_pop.drop_duplicates(subset = "city_state")
list(city_pop)
city_pop #760

,place_code,city,state,pop_estimate_2021,pop_estimate_2020,state_code,city_state,city_state_code,place_state_code
0,3076,Auburn,Alabama,78564,76429,AL,"Auburn, Alabama","Auburn, AL",3076AL
1,7000,Birmingham,Alabama,197575,200133,AL,"Birmingham, Alabama","Birmingham, AL",7000AL
2,20104,Decatur,Alabama,57804,57777,AL,"Decatur, Alabama","Decatur, AL",20104AL
3,21184,Dothan,Alabama,71175,71154,AL,"Dothan, Alabama","Dothan, AL",21184AL
4,35896,Hoover,Alabama,92589,92687,AL,"Hoover, Alabama","Hoover, AL",35896AL
...,...,...,...,...,...,...,...,...,...
1833,66000,Racine,Wisconsin,77127,77578,WI,"Racine, Wisconsin","Racine, WI",66000WI
1834,84250,Waukesha,Wisconsin,71256,71365,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI
1835,85300,West Allis,Wisconsin,59484,60223,WI,"West Allis, Wisconsin","West Allis, WI",85300WI
1872,13150,Casper,Wyoming,58656,59242,WY,"Casper, Wyoming","Casper, WY",13150WY


In [59]:
city_land = pd.read_csv("2020_Gaz_place_national.txt",sep = "\t",encoding='latin-1')
state_codes = pd.read_csv("state_codes.csv")
city_land
city_only = [i.lower().__contains__("city") for i in city_land["NAME"]]
city_land= city_land[city_only]
city_land = city_land[["USPS","GEOID","NAME","ALAND"]].set_axis(['state_code', 'geo_code', 'city', 'land_area_sqm'], axis='columns', copy = False)

state_codes = state_codes[["state","code"]].set_axis(['state', 'state_code'], axis='columns', copy = False)
city_land = pd.merge(city_land, state_codes, on = "state_code", how = "left")
city_land["city"] = [i.replace(" city","") for i in city_land["city"]]
city_land["city_state"] = city_land["city"] + ", " + city_land["state"]
city_land["city_state_code"] = city_land["city"] + ", " + city_land["state_code"]
city_land["land_area_sqkm"] = city_land["land_area_sqm"]*0.000001
city_land["geo_code"] = [str(i) for i in city_land["geo_code"]]
city_land #10512

,state_code,geo_code,city,land_area_sqm,state,city_state,city_state_code,land_area_sqkm
0,AL,100124,Abbeville,40255362,Alabama,"Abbeville, Alabama","Abbeville, AL",40.255362
1,AL,100460,Adamsville,67895675,Alabama,"Adamsville, Alabama","Adamsville, AL",67.895675
2,AL,100820,Alabaster,65472841,Alabama,"Alabaster, Alabama","Alabaster, AL",65.472841
3,AL,100988,Albertville,69770754,Alabama,"Albertville, Alabama","Albertville, AL",69.770754
4,AL,101132,Alexander City,113186307,Alabama,"Alexander City, Alabama","Alexander City, AL",113.186307
...,...,...,...,...,...,...,...,...
10507,WY,5666220,Riverton,26780417,Wyoming,"Riverton, Wyoming","Riverton, WY",26.780417
10508,WY,5667235,Rock Springs,51160520,Wyoming,"Rock Springs, Wyoming","Rock Springs, WY",51.160520
10509,WY,5669845,Sheridan,32850653,Wyoming,"Sheridan, Wyoming","Sheridan, WY",32.850653
10510,WY,5677530,Torrington,9644842,Wyoming,"Torrington, Wyoming","Torrington, WY",9.644842


In [60]:
walking_data = pd.merge(city_pop,city_land[["city_state", "geo_code", "land_area_sqkm"]], on = "city_state", how = "left")
walking_data["pop_per_km2"] = walking_data["pop_estimate_2021"]/walking_data["land_area_sqkm"]
walking_data #760

,place_code,city,state,pop_estimate_2021,pop_estimate_2020,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,pop_per_km2
0,3076,Auburn,Alabama,78564,76429,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,500.137630
1,7000,Birmingham,Alabama,197575,200133,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,518.840515
2,20104,Decatur,Alabama,57804,57777,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,410.581694
3,21184,Dothan,Alabama,71175,71154,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,305.930918
4,35896,Hoover,Alabama,92589,92687,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,743.742746
...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,77578,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,1924.323130
756,84250,Waukesha,Wisconsin,71256,71365,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,1077.955667
757,85300,West Allis,Wisconsin,59484,60223,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,2017.954722
758,13150,Casper,Wyoming,58656,59242,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,853.090863


In [61]:
county = pd.read_csv("city_county.csv")
county["county_state"] = county["county"] + ", " + county["state"]
walking_data = pd.merge(walking_data, county[["county_state","city_state_code"]], on = "city_state_code", how = "left")
walking_data

,place_code,city,state,pop_estimate_2021,pop_estimate_2020,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,pop_per_km2,county_state
0,3076,Auburn,Alabama,78564,76429,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,500.137630,"Lee, Alabama"
1,7000,Birmingham,Alabama,197575,200133,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,518.840515,"Jefferson, Alabama"
2,20104,Decatur,Alabama,57804,57777,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,410.581694,"Morgan, Alabama"
3,21184,Dothan,Alabama,71175,71154,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,305.930918,"Houston County, Alabama"
4,35896,Hoover,Alabama,92589,92687,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,743.742746,"Jefferson, Alabama"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,77578,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,1924.323130,"Racine, Wisconsin"
756,84250,Waukesha,Wisconsin,71256,71365,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,1077.955667,"Waukesha County, Wisconsin"
757,85300,West Allis,Wisconsin,59484,60223,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,2017.954722,"Milwaukee, Wisconsin"
758,13150,Casper,Wyoming,58656,59242,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,853.090863,"Natrona County, Wyoming"


In [62]:
age_sex_columns = pd.read_csv("Age_and_Sex_2021-Column-Metadata.csv",encoding='latin-1')
age_sex = pd.read_csv("Age_and_Sex_2021-Data.csv",encoding='latin-1')
del_moe = [not i.__contains__("Margin of Error") for i in age_sex_columns["Label"]]
age_sex_columns = age_sex_columns[del_moe]
del_ann = [not i.__contains__("A") for i in age_sex_columns["Column Name"]]
age_sex_columns = age_sex_columns[del_ann]
age_label = [i.split("!!")[-1] for i in age_sex_columns["Label"].iloc[2:20]]
age_label = [i.lower().replace(" ","_") for i in age_label]
age_label
age_col = age_sex_columns["Column Name"].iloc[2:20]
age_col
sex_col = ["S0101_C03_001E","S0101_C05_001E"] 
sex_label = ["male","female"]


city_only = [i.lower().__contains__("city") for i in age_sex["NAME"]]
age_sex= age_sex[city_only]
age_sex["city_state"] = [i.replace(" city","") for i in age_sex["NAME"]]


age = age_sex[age_col].set_axis(age_label, axis='columns', copy = False)
age["city_state"] = age_sex["city_state"]
age

sex = age_sex[sex_col].set_axis(sex_label, axis='columns', copy = False)
sex["city_state"] = age_sex["city_state"]
sex

walking_data = pd.merge(walking_data, age, on = "city_state", how = "left")
walking_data = pd.merge(walking_data, sex, on = "city_state", how = "left")

walking_data


,place_code,city,state,pop_estimate_2021,pop_estimate_2020,state_code,city_state,city_state_code,place_state_code,geo_code,...,50_to_54_years,55_to_59_years,60_to_64_years,65_to_69_years,70_to_74_years,75_to_79_years,80_to_84_years,85_years_and_over,male,female
0,3076,Auburn,Alabama,78564,76429,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,...,3108,3030,3107,2847,2489,2511,594,534,37961,40591
1,7000,Birmingham,Alabama,197575,200133,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,...,10264,10336,13019,13470,8671,4777,3161,3415,94696,101714
2,20104,Decatur,Alabama,57804,57777,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21184,Dothan,Alabama,71175,71154,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,...,4581,4316,4973,3733,4326,2263,1620,1637,33706,37577
4,35896,Hoover,Alabama,92589,92687,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,...,5046,5454,5982,3585,3833,2536,2917,1948,46671,45917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,77578,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,...,3968,3564,4923,3004,3061,2039,950,945,36786,40345
756,84250,Waukesha,Wisconsin,71256,71365,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,...,3844,5355,4569,3843,2593,1684,1385,1419,34392,36862
757,85300,West Allis,Wisconsin,59484,60223,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,13150,Casper,Wyoming,58656,59242,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
race = pd.read_csv("Race_2020.p1-Data.csv",encoding='latin-1')
race_columns = pd.read_csv("Race_2020.p1-Column-Metadata.csv",encoding='latin-1')
race
race_columns = race_columns[[ not i.__contains__("A") for i in race_columns["Column Name"]]]
more_col = pd.Series(["P1_009N"])
more_label = ["two_or_more_races"]
temp = race_columns[[i.__contains__("alone") for i in race_columns["Label"]]]
alone_col = temp["Column Name"]
alone_label = [i.split("!!")[-1].lower().replace(" ","_") for i in temp["Label"]]

col = pd.concat([pd.Series("NAME"),alone_col,more_col])
label = ["city_state"] + alone_label + more_label

race = race[[i.__contains__("city") for i in race["NAME"]]]
race["NAME"] = [i.replace(" city", "") for i in race["NAME"]]
race = race[col].set_axis(label,axis="columns",copy=False)

walking_data = pd.merge(walking_data, race, on = "city_state", how = "left")

walking_data

race

,city_state,white_alone,black_or_african_american_alone,american_indian_and_alaska_native_alone,asian_alone,native_hawaiian_and_other_pacific_islander_alone,some_other_race_alone,two_or_more_races
1,"Abbeville, Alabama",1165,1039,5,15,0,51,83
2,"Adamsville, Alabama",1741,2313,31,10,3,123,145
5,"Alabaster, Alabama",22323,5356,232,396,28,2416,2533
6,"Albertville, Alabama",13830,1095,531,190,15,4841,1884
7,"Alexander City, Alabama",8724,4911,56,156,0,464,532
...,...,...,...,...,...,...,...,...
31572,"Riverton, Wyoming",7836,100,1521,63,8,323,831
31575,"Rock Springs, Wyoming",18865,270,243,211,42,1533,2362
31580,"Sheridan, Wyoming",16735,78,217,131,14,356,1206
31598,"Torrington, Wyoming",5272,63,71,38,1,281,393


In [64]:
hisp = pd.read_csv("Hispanic_2020.P2-Data.csv",encoding='latin-1', low_memory=False)
hisp_col = ["NAME","P2_002N"]
hisp_label = ["city_state","hispanic_or_latino"]
hisp = hisp[[i.__contains__("city") for i in hisp["NAME"]]]
hisp["NAME"] = [i.replace(" city","") for i in hisp["NAME"]]
hisp = hisp[hisp_col].set_axis(hisp_label, axis="columns", copy=False)

walking_data = pd.merge(walking_data, hisp, on = "city_state", how = "left")
walking_data

,place_code,city,state,pop_estimate_2021,pop_estimate_2020,state_code,city_state,city_state_code,place_state_code,geo_code,...,male,female,white_alone,black_or_african_american_alone,american_indian_and_alaska_native_alone,asian_alone,native_hawaiian_and_other_pacific_islander_alone,some_other_race_alone,two_or_more_races,hispanic_or_latino
0,3076,Auburn,Alabama,78564,76429,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,...,37961,40591,48918.0,14009.0,238.0,7414.0,38.0,1597.0,3929.0,3619
1,7000,Birmingham,Alabama,197575,200133,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,...,94696,101714,47209.0,137296.0,755.0,3275.0,123.0,5360.0,6715.0,9274
2,20104,Decatur,Alabama,57804,57777,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,...,NaN,NaN,33190.0,13441.0,433.0,531.0,56.0,5983.0,4304.0,9070
3,21184,Dothan,Alabama,71175,71154,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,...,33706,37577,40496.0,23896.0,285.0,1141.0,47.0,1444.0,3763.0,3131
4,35896,Hoover,Alabama,92589,92687,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,...,46671,45917,63664.0,15621.0,254.0,5929.0,20.0,1940.0,5178.0,4704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,77578,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,...,36786,40345,40048.0,18618.0,547.0,601.0,23.0,8077.0,9902.0,18856
756,84250,Waukesha,Wisconsin,71256,71365,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,...,34392,36862,55908.0,2466.0,373.0,2404.0,41.0,3162.0,6804.0,9979
757,85300,West Allis,Wisconsin,59484,60223,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,...,NaN,NaN,44286.0,3818.0,787.0,1545.0,23.0,3615.0,6251.0,10211
758,13150,Casper,Wyoming,58656,59242,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,...,NaN,NaN,50857.0,672.0,804.0,531.0,59.0,1653.0,4462.0,5582


In [65]:
health = pd.read_csv("PLACES__Local_Data_for_Better_Health__County_Data_2022_release.csv",encoding='latin-1')
health["MeasureId"] = [i.lower() for i in health["MeasureId"]]
health[["Measure", "MeasureId"]].drop_duplicates().to_csv("health_variable_key.csv",index = False)

health["city_state"] =  health["LocationName"] + ", " + health["StateDesc"]
health = health[["city_state","MeasureId","Data_Value"]]

health = health.pivot_table(values='Data_Value', index=["city_state"], columns="MeasureId")

walking_data = pd.merge(walking_data, health, on = "city_state", how = "left")
walking_data




,place_code,city,state,pop_estimate_2021,pop_estimate_2020,state_code,city_state,city_state_code,place_state_code,geo_code,...,highchol,kidney,lpa,mammouse,mhlth,obesity,phlth,sleep,stroke,teethlost
0,3076,Auburn,Alabama,78564,76429,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7000,Birmingham,Alabama,197575,200133,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20104,Decatur,Alabama,57804,57777,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21184,Dothan,Alabama,71175,71154,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,35896,Hoover,Alabama,92589,92687,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,77578,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,...,32.55,2.90,21.70,76.0,13.7,36.10,10.15,32.90,3.0,10.2
756,84250,Waukesha,Wisconsin,71256,71365,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,...,30.60,2.45,16.25,76.7,11.7,28.95,7.70,28.15,2.3,6.6
757,85300,West Allis,Wisconsin,59484,60223,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,13150,Casper,Wyoming,58656,59242,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
# covid = pd.read_csv("epidemiology.csv",encoding='latin-1')
# covid = covid[[str(i)[0:2]=="US" for i in covid["location_key"]]]
# covid.to_csv("covid_US.csv", index = False)

covid = pd.read_csv("covid_US.csv",encoding='latin-1')
covid = covid.dropna(subset=["cumulative_confirmed","cumulative_deceased"])
#covid.pivot_table(values=["date","cumulative_confirmed","cumulative_deceased"], index="location_key",aggfunc={"date":np.max,"cumulative_confirmed":np.max,"cumulative_deceased": np.max})
covid = covid[[i=="2022-05-13" for i in covid["date"]]]
covid = covid[["date","location_key","cumulative_confirmed","cumulative_deceased"]]

covid_index = pd.read_csv("covid_index.csv",encoding='latin-1')
covid_index = covid_index[covid_index["country_code"]=="US"]
covid_index["county_state"] = covid_index["subregion2_name"]+ ", " + covid_index["subregion1_name"]
covid = pd.merge(covid,covid_index[["county_state", "location_key"]],on = "location_key", how = "left")
walking_data = pd.merge(walking_data, covid[["county_state","cumulative_confirmed","cumulative_deceased"]], on = "county_state", how = "left")
walking_data

,place_code,city,state,pop_estimate_2021,pop_estimate_2020,state_code,city_state,city_state_code,place_state_code,geo_code,...,lpa,mammouse,mhlth,obesity,phlth,sleep,stroke,teethlost,cumulative_confirmed,cumulative_deceased
0,3076,Auburn,Alabama,78564,76429,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7000,Birmingham,Alabama,197575,200133,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20104,Decatur,Alabama,57804,57777,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21184,Dothan,Alabama,71175,71154,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25800.0,506.0
4,35896,Hoover,Alabama,92589,92687,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,77578,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,...,21.70,76.0,13.7,36.10,10.15,32.90,3.0,10.2,NaN,NaN
756,84250,Waukesha,Wisconsin,71256,71365,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,...,16.25,76.7,11.7,28.95,7.70,28.15,2.3,6.6,115887.0,1093.0
757,85300,West Allis,Wisconsin,59484,60223,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,13150,Casper,Wyoming,58656,59242,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22380.0,273.0


Add more code here ^. Use variable "walking_data" add more variables to the dataset. The codes below will upload "walking_data" to the csv and output the results. Do NOT edit any previous code. Just add on.

In [67]:
walking_data.to_csv("walkable-cities.csv", index = False)

In [68]:
walkable_cities = pd.read_csv("walkable-cities.csv")
walkable_cities

,place_code,city,state,pop_estimate_2021,pop_estimate_2020,state_code,city_state,city_state_code,place_state_code,geo_code,...,lpa,mammouse,mhlth,obesity,phlth,sleep,stroke,teethlost,cumulative_confirmed,cumulative_deceased
0,3076,Auburn,Alabama,78564,76429,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7000,Birmingham,Alabama,197575,200133,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20104,Decatur,Alabama,57804,57777,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21184,Dothan,Alabama,71175,71154,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25800.0,506.0
4,35896,Hoover,Alabama,92589,92687,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,77578,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000.0,...,21.70,76.0,13.7,36.10,10.15,32.90,3.0,10.2,NaN,NaN
756,84250,Waukesha,Wisconsin,71256,71365,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250.0,...,16.25,76.7,11.7,28.95,7.70,28.15,2.3,6.6,115887.0,1093.0
757,85300,West Allis,Wisconsin,59484,60223,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,13150,Casper,Wyoming,58656,59242,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22380.0,273.0
